In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

c:\Users\SALAH\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [14]:

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [18]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device=x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Take the output from the last time step
        out = self.fc(out)
        return out




In [16]:
# Load data
train_dataset = datasets.MNIST(root=r'D:\github repos\pytorch learning\Pytorch_Learning\PyTorch tutorials\00-quickstart/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [12]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
accuracy_list = []

In [25]:
for epoch in range(num_epochs):
    print(f"Epoch: {epoch+1}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Reshape the data to fit the RNN input format
        data = data[:, 0, :, :]  # take only one channel (MNIST is grayscale)
        data = data.view(-1, sequence_length, input_size).to(device=device)
        targets = targets.to(device=device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Check accuracy after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in train_loader:
            data = data[:, 0, :, :]  
            data = data.to(device=device)
            targets = targets.to(device=device)

            scores = model(data)
            _, predicted = torch.max(scores, dim=1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

Epoch: 1
Epoch: 2


In [27]:
accuracy = correct / total
accuracy_list.append(accuracy)
print(f"Accuracy on epoch {epoch+1}: {accuracy}")

Accuracy on epoch 2: 0.8576166666666667
